# Submit

## Import

In [1]:
## Allow more than one output for a single code cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import pandas as pd
import scipy.sparse as sps
import numpy as np
import os

from skopt.space import Real, Integer, Categorical

## Set the numpy random seed
SEED = 42
np.random.seed(SEED)

os.getcwd()

'/home/jupyter/RecSysChallenge2021'

In [3]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

from Evaluation.Evaluator import EvaluatorHoldout

from Recommenders.Recommender_import_list import *
from Recommenders.Hybrid.DifferentLossScoresHybridRecommender import DifferentLossScoresHybridRecommender

from Recommenders.DataIO import DataIO

In [4]:
## Utility Functions
from Dataset.load_data import load_data
from Dataset.write_submission import write_submission
from Dataset.load_test_user_array import load_test_user_array

## Data Loading and Split

In [5]:
URM_all, ICM_dict = load_data()

In [6]:
URM_aug_all = sps.vstack([URM_all.copy().tocoo(), 
                          #ICM_dict['ICM_genre'].T.tocoo(),
                          ICM_dict['ICM_subgenre'].T.tocoo(), 
                          #ICM_dict['ICM_event'].T.tocoo(), 
                          ICM_dict['ICM_channel'].T.tocoo()], format='csr')

In [7]:
test_UserID_array = load_test_user_array()

## RP3 beta

In [8]:
"""
RP3beta = RP3betaRecommender(URM_aug_all)
RP3beta.fit(topK = 51, alpha = 0.620106, beta = 0.566617, normalize_similarity = True)

popular_items = RP3beta.recommend(test_UserID_array,
                                  cutoff = 10, 
                                  remove_seen_flag = True, 
                                  items_to_compute = None,
                                  remove_top_pop_flag = False,
                                  remove_custom_items_flag = False,
                                  return_scores = False)

write_submission(popular_items, filename = 'RP3beta_excludeSeen_submission.csv')
"""

"\nRP3beta = RP3betaRecommender(URM_aug_all)\nRP3beta.fit(topK = 51, alpha = 0.620106, beta = 0.566617, normalize_similarity = True)\n\npopular_items = RP3beta.recommend(test_UserID_array,\n                                  cutoff = 10, \n                                  remove_seen_flag = True, \n                                  items_to_compute = None,\n                                  remove_top_pop_flag = False,\n                                  remove_custom_items_flag = False,\n                                  return_scores = False)\n\nwrite_submission(popular_items, filename = 'RP3beta_excludeSeen_submission.csv')\n"

## SLIM ElasticNet

In [9]:
SLIM_BPR = SLIM_BPR_Cython(URM_aug_all)
SLIM_BPR.fit(topK = 262, epochs = 315, symmetric = True, lambda_i = 0.000022, lambda_j = 0.000333, learning_rate = 0.037189, sgd_mode='adagrad',
             positive_threshold_BPR = None,
             train_with_sparse_weights = None,
             allow_train_with_sparse_weights = False)

SLIM_BPR_Recommender: Automatic selection of fastest train mode. Available RAM is 11420.00 MB (71.19%) of 16041.00 MB, required is 1304.51 MB. Using dense matrix.
Processed 13976 (100.0%) in 0.71 sec. BPR loss is 2.37E+01. Sample per second: 19612
SLIM_BPR_Recommender: Epoch 1 of 315. Elapsed time 0.63 sec
Processed 13976 (100.0%) in 1.33 sec. BPR loss is 4.59E+01. Sample per second: 10508
SLIM_BPR_Recommender: Epoch 2 of 315. Elapsed time 1.24 sec
Processed 13976 (100.0%) in 0.94 sec. BPR loss is 5.44E+01. Sample per second: 14838
SLIM_BPR_Recommender: Epoch 3 of 315. Elapsed time 1.85 sec
Processed 13976 (100.0%) in 1.55 sec. BPR loss is 5.74E+01. Sample per second: 9008
SLIM_BPR_Recommender: Epoch 4 of 315. Elapsed time 2.46 sec
Processed 13976 (100.0%) in 1.18 sec. BPR loss is 6.21E+01. Sample per second: 11854
SLIM_BPR_Recommender: Epoch 5 of 315. Elapsed time 3.09 sec
Processed 13976 (100.0%) in 0.81 sec. BPR loss is 6.21E+01. Sample per second: 17288
SLIM_BPR_Recommender: Epoch 

In [10]:
SLIMElasticNet = MultiThreadSLIM_SLIMElasticNetRecommender(URM_aug_all)
#SLIMElasticNet.load_model(folder_path = 'result_experiments/SLIM_elasticNet_AUG_subgenre_channel_topK_categorical/', file_name = 'SLIMElasticNetRecommender_best_model.zip')
SLIMElasticNet.fit(topK = 673, l1_ratio = 0.001155, alpha = 0.073102)

100%|█████████▉| 18032/18059 [30:16<00:02, 12.02it/s]

In [11]:
DiffLoss = DifferentLossScoresHybridRecommender(URM_aug_all, [SLIMElasticNet, SLIM_BPR])
DiffLoss.fit(norm = np.inf, alpha = 0.826909)
print("Fitting ended.")

Fitting ended.


In [12]:
IALS = IALSRecommender(URM_aug_all)
IALS.fit(num_factors = 52, epochs = 15, confidence_scaling = 'linear', alpha = 1.3346069348060516, epsilon = 0.17158990394166584, reg = 0.0005841170332713697)

100%|█████████▉| 18056/18059 [30:30<00:00, 12.02it/s]

IALSRecommender: Epoch 1 of 15. Elapsed time 14.08 sec
IALSRecommender: Epoch 2 of 15. Elapsed time 27.63 sec
IALSRecommender: Epoch 3 of 15. Elapsed time 40.90 sec
IALSRecommender: Epoch 4 of 15. Elapsed time 54.26 sec
IALSRecommender: Epoch 5 of 15. Elapsed time 1.13 min
IALSRecommender: Epoch 6 of 15. Elapsed time 1.37 min
IALSRecommender: Epoch 7 of 15. Elapsed time 1.59 min
IALSRecommender: Epoch 8 of 15. Elapsed time 1.81 min
IALSRecommender: Epoch 9 of 15. Elapsed time 2.08 min
IALSRecommender: Epoch 10 of 15. Elapsed time 2.32 min
IALSRecommender: Epoch 11 of 15. Elapsed time 2.54 min
IALSRecommender: Epoch 12 of 15. Elapsed time 2.77 min
IALSRecommender: Epoch 13 of 15. Elapsed time 3.00 min
IALSRecommender: Epoch 14 of 15. Elapsed time 3.23 min
IALSRecommender: Epoch 15 of 15. Elapsed time 3.46 min
IALSRecommender: Terminating at epoch 15. Elapsed time 3.46 min


In [13]:
model = DifferentLossScoresHybridRecommender(URM_aug_all, [DiffLoss, IALS])
model.fit(norm = 1, alpha = 0.8390415906812541)
print("Fitting ended.")

Fitting ended.


In [14]:
popular_items = model.recommend(test_UserID_array,
                                   cutoff = 10, 
                                   remove_seen_flag = True, 
                                   items_to_compute = None,
                                   remove_top_pop_flag = False,
                                   remove_custom_items_flag = False,
                                   return_scores = False)

write_submission(popular_items, filename = 'DiffLoss_DiffLossSLIM_IALS_excludeSeen_submission.csv')